<a href="https://colab.research.google.com/github/SeishiroInoue/SeishiroInoue/blob/main/research7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **メルカリ商品データの分析**

## **2値分類**

* 商品が売れていないか（販売中）、売れたか（売り切れ）

変数は

* 目的変数: 商品状態
* 説明変数: 販売価格、カテゴリID、商品の状態、送料の負担、発送の方法、発送元の地域、発送にかかる日数、商品名の長さ、商品説明文の長さ

アルゴリズム（戦略）は

* ランダムフォレスト

を用いる

より高い精度を目指す

## **1. データセットの説明**

フリマアプリ「メルカリ」上に公開している商品データ

* データ抽出日 : 2022年07月16日
* データ対象期間 : 2020年01月01日〜2020年12月31日の間に出品された商品
* 件数 : 144,674,037 件
* データサイズ : 約 100 GB

## **2. データの準備**

516個あるcsvファイルのうち、初めの5
個（mercari_items_202208_000000000000.csv ~ mercari_items_202208_000000000004.csv）を使う

In [ ]:
# ライブラリをインポート
import pandas as pd
import glob

# csvファイルのパスを取得
file_paths = glob.glob('/Users/inoue/documents/特別研究/データ/items/*.csv')

# 空のデータフレームを作成
combined_df = pd.DataFrame()

# csvファイルを読み込んで結合
for file_path in file_paths:
    df = pd.read_csv(file_path)
    combined_df = combined_df.append(df, ignore_index=True)

# 結合したデータフレームの冒頭5行をを表示
combined_df.head()

/var/folders/j8/kt4l6ys96jn8w516h8hpp3w80000gn/T/ipykernel_24086/2015087067.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
/var/folders/j8/kt4l6ys96jn8w516h8hpp3w80000gn/T/ipykernel_24086/2015087067.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
/var/folders/j8/kt4l6ys96jn8w516h8hpp3w80000gn/T/ipykernel_24086/2015087067.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
/var/folders/j8/kt4l6ys96jn8w516h8hpp3w80000gn/T/ipykernel_24086/2015087067.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a futur

,anon_item_id,status,name,description,price,category_id,item_condition,size,brand_name,shipping_payer,shipping_method,shipping_from_area,shipping_duration,num_likes,num_comments,updated,created
0,3afc6b86-0933-4f75-a3d5-b124d7707792,sold_out,【新品】WAIPOUA ワイポウア H81 メリノウール ライトロングTシャツ,"ご覧いただきありがとうございます。\n\n値下げしました ¥7,700 → ¥6,500...",6500,120,1,3.0,NaN,2,14,20,3,4,0,2020-11-08 22:54:15,2020-01-03 03:58:48
1,1309267d-632b-47b0-b1b8-b9efae32aeab,sold_out,ミレニアム 1 [下] (ドラゴン・タトゥーの女 下),「ミレニアム 1〔下〕」\nスティーグ・ラーソン\n定価: ￥ 880\n\n#スティーグ・...,333,668,3,NaN,NaN,2,17,23,2,0,0,2020-02-15 10:13:36,2020-01-03 03:58:48
2,51cedc1f-3b8b-45b7-922a-9563f05d6803,sold_out,ワコール 産前産後骨盤ベルト,他の骨盤ベルトを使用した為出品します。\n\nカラー:黒\nサイズ:M\n\n※箱無し対応可...,3950,293,1,3.0,1389.0,2,9,13,2,1,0,2020-01-09 04:50:09,2020-01-03 03:58:49
3,dde25cd3-fb93-46e1-802a-462452292773,sold_out,s★chan様専用 BIBS Pacifier vanilla,vanilla\n0-6months,1200,526,1,NaN,NaN,2,17,14,2,0,0,2020-01-10 09:43:13,2020-01-03 03:58:49
4,cd4c4fe3-490c-4119-b385-ecee905d2c53,sold_out,城北高等学校 5年間スーパー過去問 2019年度用,"「城北高等学校 5年間スーパー過去問」\n定価: ￥ 2,310\n\n子ども用に購入しまし...",800,1124,3,NaN,NaN,2,14,13,1,1,0,2020-01-26 14:59:13,2020-01-03 03:58:49


In [ ]:
# レコード数を表示
len(combined_df)

1638642

## **3. データの加工**

In [ ]:
# 必要な列のデータのみ抽出
required_df = combined_df[['status', 'name', 'description', 'price', 'category_id', 'item_condition', 'shipping_payer', 'shipping_method', 'shipping_from_area', 'shipping_duration']]
required_df.head()

,status,name,description,price,category_id,item_condition,shipping_payer,shipping_method,shipping_from_area,shipping_duration
0,sold_out,【新品】WAIPOUA ワイポウア H81 メリノウール ライトロングTシャツ,"ご覧いただきありがとうございます。\n\n値下げしました ¥7,700 → ¥6,500...",6500,120,1,2,14,20,3
1,sold_out,ミレニアム 1 [下] (ドラゴン・タトゥーの女 下),「ミレニアム 1〔下〕」\nスティーグ・ラーソン\n定価: ￥ 880\n\n#スティーグ・...,333,668,3,2,17,23,2
2,sold_out,ワコール 産前産後骨盤ベルト,他の骨盤ベルトを使用した為出品します。\n\nカラー:黒\nサイズ:M\n\n※箱無し対応可...,3950,293,1,2,9,13,2
3,sold_out,s★chan様専用 BIBS Pacifier vanilla,vanilla\n0-6months,1200,526,1,2,17,14,2
4,sold_out,城北高等学校 5年間スーパー過去問 2019年度用,"「城北高等学校 5年間スーパー過去問」\n定価: ￥ 2,310\n\n子ども用に購入しまし...",800,1124,3,2,14,13,1


商品名の長さ、商品説明文の長さを表す'name_length'、'description_length'カラムを追加

In [ ]:
# 改行コードと空白を削除
required_df = required_df.replace('\n', '', regex=True)
required_df = required_df.replace(' ', '', regex=True).replace('　', '', regex=True)

# 'name_length'、'description_length'カラムを追加
required_df['name_length'] = list(map(len, required_df['name']))
required_df['description_length'] = list(map(len, required_df['description']))

# 'name'、'description'カラムを削除
required_df = required_df.drop('name', axis=1)
required_df = required_df.drop('description', axis=1)

# 冒頭5行をを表示
required_df.head()

,status,price,category_id,item_condition,shipping_payer,shipping_method,shipping_from_area,shipping_duration,name_length,description_length
0,sold_out,6500,120,1,2,14,20,3,35,555
1,sold_out,333,668,3,2,17,23,2,23,250
2,sold_out,3950,293,1,2,9,13,2,13,73
3,sold_out,1200,526,1,2,17,14,2,28,16
4,sold_out,800,1124,3,2,14,13,1,23,105


「取引中」のデータも売れていることは確かなので、以後「売り切れ」のデータと合算して分析する

* 出品中: 0
* 取引中、売り切れ: 1

として、ダミー変数を定義する

In [ ]:
# ダミー変数を定義
required_df['status'] = required_df['status'].map({'on_sale': 0, 'trading': 1, 'sold_out': 1})
required_df.head()

,status,price,category_id,item_condition,shipping_payer,shipping_method,shipping_from_area,shipping_duration,name_length,description_length
0,1,6500,120,1,2,14,20,3,35,555
1,1,333,668,3,2,17,23,2,23,250
2,1,3950,293,1,2,9,13,2,13,73
3,1,1200,526,1,2,17,14,2,28,16
4,1,800,1124,3,2,14,13,1,23,105


In [ ]:
# 売れていない件数を表示
len(required_df[required_df['status'] == 0])

243194

In [ ]:
# 売れた件数を表示
len(required_df[required_df['status'] == 1])

1395448

## **4. モデルの学習・検証**

In [ ]:
# ライブラリをインポート
from sklearn.model_selection import train_test_split

# 説明変数と目的変数を定義
X = required_df.drop('status', axis=1)
y = required_df['status']

# 訓練データ(80%)とテストデータ(20%)に分ける
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 次元数を表示
X_train.shape, X_test.shape

((1310913, 9), (327729, 9))

評価・検証ライブラリ

In [ ]:
# ライブラリをインポート
from sklearn.metrics import accuracy_score

組み合わせライブラリ

In [ ]:
# ライブラリをインポート
from itertools import combinations

## **ランダムフォレスト**

複数の決定木を組み合わせて予測する手法

In [ ]:
# ライブラリをインポート
from sklearn.ensemble import RandomForestClassifier

# モデルのインスタンスを作成
rf = RandomForestClassifier(random_state=42)

# モデルの訓練
rf.fit(X_train, y_train)

# テストデータでの精度を計算
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# 精度を表示
print('正解率:', accuracy)

正解率: 0.8690350869163241


## **5. 欠損値の補完**

* サイズ
* ブランド名

の2つが欠損値となっている

In [ ]:
# 必要な列のデータのみ抽出
complemented_df = combined_df[['status', 'name', 'description', 'price', 'category_id', 'item_condition', 'size', 'brand_name', 'shipping_payer', 'shipping_method', 'shipping_from_area', 'shipping_duration']]
complemented_df.head()

,status,name,description,price,category_id,item_condition,size,brand_name,shipping_payer,shipping_method,shipping_from_area,shipping_duration
0,sold_out,【新品】WAIPOUA ワイポウア H81 メリノウール ライトロングTシャツ,"ご覧いただきありがとうございます。\n\n値下げしました ¥7,700 → ¥6,500...",6500,120,1,3.0,NaN,2,14,20,3
1,sold_out,ミレニアム 1 [下] (ドラゴン・タトゥーの女 下),「ミレニアム 1〔下〕」\nスティーグ・ラーソン\n定価: ￥ 880\n\n#スティーグ・...,333,668,3,NaN,NaN,2,17,23,2
2,sold_out,ワコール 産前産後骨盤ベルト,他の骨盤ベルトを使用した為出品します。\n\nカラー:黒\nサイズ:M\n\n※箱無し対応可...,3950,293,1,3.0,1389.0,2,9,13,2
3,sold_out,s★chan様専用 BIBS Pacifier vanilla,vanilla\n0-6months,1200,526,1,NaN,NaN,2,17,14,2
4,sold_out,城北高等学校 5年間スーパー過去問 2019年度用,"「城北高等学校 5年間スーパー過去問」\n定価: ￥ 2,310\n\n子ども用に購入しまし...",800,1124,3,NaN,NaN,2,14,13,1


In [ ]:
# サイズとブランド名の欠損値を数える
missing_values_count_1 = complemented_df['size'].isna().sum()
missing_values_count_2 = complemented_df['brand_name'].isna().sum()

print('サイズの欠損値の数:', missing_values_count_1)
print('ブランド名の数:', missing_values_count_2)

サイズの欠損値の数: 1286478
ブランド名の数: 1080907


In [ ]:
# サイズの欠損していない値を数える
non_missing_values_count_1 = complemented_df['size'].notna().sum()
non_missing_values_count_2 = complemented_df['brand_name'].notna().sum()

print('サイズの非欠損値の数:', non_missing_values_count_1)
print('ブランド名の非欠損値の数:', non_missing_values_count_2)

サイズの非欠損値の数: 352164
ブランド名の非欠損値の数: 557735


In [ ]:
# 改行コードと空白を削除
complemented_df = complemented_df.replace('\n', '', regex=True)
complemented_df = complemented_df.replace(' ', '', regex=True).replace('　', '', regex=True)

# 'name_length'、'description_length'カラムを追加
complemented_df['name_length'] = list(map(len, complemented_df['name']))
complemented_df['description_length'] = list(map(len, complemented_df['description']))

# 'name'、'description'カラムを削除
complemented_df = complemented_df.drop('name', axis=1)
complemented_df = complemented_df.drop('description', axis=1)

# 冒頭5行をを表示
complemented_df.head()

,status,price,category_id,item_condition,size,brand_name,shipping_payer,shipping_method,shipping_from_area,shipping_duration,name_length,description_length
0,sold_out,6500,120,1,3.0,NaN,2,14,20,3,35,555
1,sold_out,333,668,3,NaN,NaN,2,17,23,2,23,250
2,sold_out,3950,293,1,3.0,1389.0,2,9,13,2,13,73
3,sold_out,1200,526,1,NaN,NaN,2,17,14,2,28,16
4,sold_out,800,1124,3,NaN,NaN,2,14,13,1,23,105


In [ ]:
# ダミー変数を定義
complemented_df['status'] = complemented_df['status'].map({'on_sale': 0, 'trading': 1, 'sold_out': 1})
complemented_df.head()

,status,price,category_id,item_condition,size,brand_name,shipping_payer,shipping_method,shipping_from_area,shipping_duration,name_length,description_length
0,1,6500,120,1,3.0,NaN,2,14,20,3,35,555
1,1,333,668,3,NaN,NaN,2,17,23,2,23,250
2,1,3950,293,1,3.0,1389.0,2,9,13,2,13,73
3,1,1200,526,1,NaN,NaN,2,17,14,2,28,16
4,1,800,1124,3,NaN,NaN,2,14,13,1,23,105


## **ロジスティック回帰で欠損値を補完**

In [ ]:
# ライブラリをインポート
from sklearn.linear_model import LinearRegression

# 欠損値の補完を行うカラムのリスト
columns_with_missing_values = ['size', 'brand_name']

for column in columns_with_missing_values:
    # 訓練データとテストデータに分ける
    train_df = complemented_df[complemented_df[column].notna()]
    test_df = complemented_df[complemented_df[column].isna()]

    # 回帰モデルの訓練
    X_train = train_df.drop(columns_with_missing_values, axis=1)
    y_train = train_df[column]
    regressor = LinearRegression().fit(X_train, y_train)

    # 欠損値の予測
    X_test = test_df.drop(columns_with_missing_values, axis=1)
    predicted_values = regressor.predict(X_test)

    # 予測値で欠損値を補完
    complemented_df.loc[complemented_df[column].isna(), column] = predicted_values

# 冒頭5行をを表示
complemented_df.head()

,status,price,category_id,item_condition,size,brand_name,shipping_payer,shipping_method,shipping_from_area,shipping_duration,name_length,description_length
0,1,6500,120,1,3.000000,3564.990258,2,14,20,3,35,555
1,1,333,668,3,34.678734,3366.425847,2,17,23,2,23,250
2,1,3950,293,1,3.000000,1389.000000,2,9,13,2,13,73
3,1,1200,526,1,27.461600,3617.954987,2,17,14,2,28,16
4,1,800,1124,3,52.740134,3800.963445,2,14,13,1,23,105


In [ ]:
# ライブラリをインポート
from sklearn.model_selection import train_test_split

# 説明変数と目的変数を定義
X = complemented_df.drop('status', axis=1)
y = complemented_df['status']

# 訓練データ(80%)とテストデータ(20%)に分ける
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 次元数を表示
X_train.shape, X_test.shape

((1310913, 11), (327729, 11))

## **ランダムフォレストで予測**

In [ ]:
# ライブラリをインポート
from sklearn.ensemble import RandomForestClassifier

# モデルのインスタンスを作成
rf = RandomForestClassifier(random_state=42)

# モデルの訓練
rf.fit(X_train, y_train)

# テストデータでの精度を計算
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# 精度を表示
print('正解率:', accuracy)

正解率: 0.9671802007146149
